# Building extraction from historical map using SVM classifier in Python


## Data Wrangling and Preparation

### Mounting Google Drive

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

### Install Dependencies

In [3]:
!pip install rasterio geopandas shapely

  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'

  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\Geomatika\miniconda3\python.exe' 'C:\Users\Geomatika\miniconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\GEOMAT~1\AppData\Local\Temp\tmpnvp1rwrx'
       cwd: C:\Users\GEOMAT~1\AppData\Local\Temp\pip-install-539juolt\rasterio
  Complete output (2 lines):
  INFO:root:Building on Windows requires extra options to setup.py to locate needed GDAL files. More information is available in the README.
  ERROR: A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
  ----------------------------------------
ERROR: Command errored out with exit status 1: 'C:\Users\Geomatika\miniconda3\python.exe' 'C:\Users\Geomatika\miniconda3\lib\site-packages\pip\_vendor\pep517\_in_process.py' get_requires_for_build_wheel 'C:\Users\GEOMAT~1\AppData\Local\Temp\tmpnvp1rwrx' Check the logs for full 


  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


### Loading libraries


In [4]:
import os
import rasterio
from rasterio.mask import mask
from skimage import io, color, exposure
import geopandas as gpd
import numpy as np
import cv2
from matplotlib import pyplot
from shapely.geometry import mapping


ModuleNotFoundError: No module named 'rasterio'

### Loading Data

In [ ]:
!wget https://drive.google.com/uc?id=1G1--4hO5vlUSH9xEGRPH_CGbTJ5XduF5 -O jogja.tif
#!wget https://github.com/danylaksono/historical-map-extraction/blob/main/Data/jogja_10000_1925.tif -O jogja.tif

!wget https://github.com/danylaksono/historical-map-extraction/blob/main/Data/clipper.zip -O clipper.zip


### Display the map

In [ ]:
from matplotlib import pyplot as plt
from rasterio.plot import show

# enlarge the plot
plt.rcParams["figure.figsize"] = (20,20)


# calling the image as source
# python 'with' ensures that the file is closed after the iteration
with rasterio.open("jogja.tif") as src:
  # checking the loaded data
  print(src.crs)
  print(src.meta)
  show(src.read(), transform=src.transform)

### Clip the map

In [ ]:
url = 'https://raw.githubusercontent.com/danylaksono/historical-map-extraction/main/Data/clipper.geojson'
shapes = gpd.read_file(url)
print(shapes.head())

In [ ]:
# clip the image
with rasterio.open("jogja.tif") as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes.geometry, crop=True)
    out_meta = src.meta
    
# update metadata for writing
out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

# write the clipped image
with rasterio.open("jogja_clipped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

# show clipped map
clippedsrc = rasterio.open("jogja_clipped.tif")
show(clippedsrc.read(), transform=clippedsrc.transform)

In [ ]:
print(clippedsrc.height)
print(clippedsrc.width)


In [ ]:
#clippedsrc.close()

## Image Classification with Machine Learning

### Preparing data

In [ ]:
import matplotlib.pyplot as plt
from rasterio.plot import show

clipped_img = clippedsrc.read([1,2,3])[:, 700:1200, 800:2000]

print(clipped_img.shape)
fig, ax = plt.subplots(figsize=(20,17))
show(clipped_img[:, :, :], ax=ax, transform=clippedsrc.transform)

In [ ]:
!wget https://github.com/danylaksono/historical-map-extraction/blob/main/Data/training.geojson -O training.geojson

In [ ]:
training = gpd.read_file('https://raw.githubusercontent.com/danylaksono/historical-map-extraction/main/Data/training.geojson')
training.crs


In [ ]:
# plot training data and image
fig, ax = plt.subplots(figsize=(15, 15))
rasterio.plot.show(clippedsrc, ax=ax)
training.plot(ax=ax, facecolor='blue', edgecolor='r');

In [ ]:
# jumlah data
len(training)

In [ ]:
X = np.array([], dtype=np.int8).reshape(0,4) # pixels for training
y = np.array([], dtype=np.string_) # labels for training

In [ ]:
# saving geometry values
geoms = training.geometry.values 

# extract the raster values within the polygon 
with rasterio.open("jogja_clipped.tif") as src:
    band_count = src.count
    for index, geom in enumerate(geoms):
        feature = [mapping(geom)]

        # the mask function returns an array of the raster pixels within this feature
        out_image, out_transform = mask(src, feature, crop=True) 
        # eliminate all the pixels with 0 values for all 8 bands - AKA not actually part of the shapefile
        out_image_trimmed = out_image[:,~np.all(out_image == 0, axis=0)]
        # eliminate all the pixels with 255 values for all 8 bands - AKA not actually part of the shapefile
        out_image_trimmed = out_image_trimmed[:,~np.all(out_image_trimmed == 255, axis=0)]
        # reshape the array to [pixel count, bands]
        out_image_reshaped = out_image_trimmed.reshape(-1, band_count)
        # append the labels to the y array
        y = np.append(y,[training["class"][index]] * out_image_reshaped.shape[0]) 
        # stack the pizels onto the pixel array
        X = np.vstack((X,out_image_reshaped))        


In [ ]:
# What are our classification labels?
labels = np.unique(training["class"])
print('The training data include {n} classes: {classes}\n'.format(n=labels.size, 
                                                                classes=labels))

# We will need a "X" matrix containing our features, and a "y" array containing our labels
print('Our X matrix is sized: {sz}'.format(sz=X.shape))
print('Our y array is sized: {sz}'.format(sz=y.shape))

### Plot to check

In [ ]:
fig, ax = plt.subplots(1,3, figsize=[20,4])

# numbers 1-8
band_count = np.arange(1,5)

classes = np.unique(y)
for class_type in classes:
    band_intensity = np.mean(X[y==class_type, :], axis=0)
    ax[0].plot(band_count, band_intensity, label=class_type)
    ax[1].plot(band_count, band_intensity, label=class_type)
    ax[2].plot(band_count, band_intensity, label=class_type)
# plot them as lines

# Add some axis labels
ax[0].set_xlabel('Band #')
ax[0].set_ylabel('Reflectance Value')
ax[1].set_ylabel('Reflectance Value')
ax[1].set_xlabel('Band #')
ax[2].set_ylabel('Reflectance Value')
ax[2].set_xlabel('Band #')
#ax[0].set_ylim(32,38)
ax[1].set_ylim(32,38)
ax[2].set_ylim(70,140)
#ax.set
ax[1].legend(loc="upper right")
# Add a title
ax[0].set_title('Band Intensities Full Overview')
ax[1].set_title('Band Intensities Lower Ref Subset')
ax[2].set_title('Band Intensities Higher Ref Subset')

### Train


In [ ]:
#from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X, y)

In [ ]:
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.windows import Window
from rasterio.plot import reshape_as_raster, reshape_as_image

In [ ]:
with rasterio.open("jogja_clipped.tif") as src:
    # may need to reduce this image size if your kernel crashes, takes a lot of memory
    img = src.read() #[:, 150:600, 250:1400]

# Take our full image and reshape into long 2d array (nrow * ncol, nband) for classification
print(img.shape)
reshaped_img = reshape_as_image(img)
print(reshaped_img.shape)

In [ ]:
class_prediction = gnb.predict(reshaped_img.reshape(-1, 4))
#class_prediction = gnb.predict(reshaped_img)
print(class_prediction)

# Reshape our classification map back into a 2D matrix so we can visualize it
class_prediction = class_prediction.reshape(reshaped_img[:, :, 0].shape)

In [ ]:
class_prediction

In [ ]:
def str_class_to_int(class_array):
    class_array[class_array == '1'] = 1
    class_array[class_array == '2'] = 2
    class_array[class_array == '3'] = 3
    class_array[class_array == '4'] = 4
    class_array[class_array == '5'] = 5
    return(class_array.astype(int))

class_prediction = str_class_to_int(class_prediction)

### Convert prediction to raster

In [ ]:
arr = class_prediction.astype('uint8')
print(arr.shape)
clippedsrc = rasterio.open("jogja_clipped.tif")

# update metadata for writing
out_meta.update({"driver": "GTiff",
                 "height": arr.shape[0],
                 "width": arr.shape[1],
                 "crs":clippedsrc.crs,
                 "transform": clippedsrc.transform})

# write the clipped image
with rasterio.open("classified.tif", "w", **out_meta) as dest:
    dest.write(arr, 1)

# show clipped map
classifiedsrc = rasterio.open("classified.tif")
show(classifiedsrc.read(), transform=classifiedsrc.transform)



In [ ]:
arr


### Convert to Polygon

In [ ]:
from shapely.geometry import Polygon
polygon = Polygon(class_prediction)
print(polygon.wkt)

### Display classification result

In [ ]:
def color_stretch(image, index):
    colors = image[:, :, index].astype(np.float64)
    for b in range(colors.shape[2]):
        colors[:, :, b] = rasterio.plot.adjust_band(colors[:, :, b])
    return colors
    
# find the highest pixel value in the prediction image
n = int(np.max(class_prediction))

# next setup a colormap for our map
colors = dict((
    (0, (48, 156, 214, 255)),   # Blue - Water
    (1, (139,69,19, 255)),      # Brown - WetSand
    (2, (96, 19, 134, 255)),    # Purple - Emergent Wetland
    (3, (244, 164, 96, 255)),   # Tan - Sand
    (4, (206, 224, 196, 255)),  # Lime - Herbaceous
    (5, (34, 139, 34, 255)),    # Forest Green - Forest 
))

# Put 0 - 255 as float 0 - 1
for k in colors:
  v = colors[k]
  _v = [_v / 255.0 for _v in v]
  colors[k] = _v
    
index_colors = [colors[key] if key in colors else 
    (255, 255, 255, 0) for key in range(0, n+1)]

cmap = plt.matplotlib.colors.ListedColormap(index_colors, 'Classification', n+1)

In [ ]:
fig, axs = plt.subplots(2,1,figsize=(50,20))

img_stretched = color_stretch(reshaped_img, [1, 2, 3])
axs[0].imshow(reshaped_img)

axs[1].imshow(class_prediction, cmap=cmap, interpolation='none')

fig.show()